In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import sys
import numpy as np
import tensorflow as tf
import tensorflow.python.platform
import random
from pathlib import Path

NUM_CLASSES = 2
IMAGE_SIZE = 28
IMAGE_PIXELS = IMAGE_SIZE*IMAGE_SIZE*3
# auto account for 70% of entile after finish parameter fiting 
TRAIN_SECTION = 6

flags = tf.app.flags
FLAGS = flags.FLAGS
tf.app.flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_string('train', 'src/data/preprocess/data_aug/train.csv', 'File name of train data')
flags.DEFINE_string('test', 'src/data/preprocess/src/test.csv', 'File name of train data')
flags.DEFINE_string('train_dir', 'src/data/preprocess/data_aug', 'Directory to put the training data.')
flags.DEFINE_integer('max_steps', 100, 'Number of steps to run trainer.')
flags.DEFINE_integer('batch_size', 4, 'Batch size'
                    'Must divide evenly into the dataset size.')
flags.DEFINE_float('learning_rate', 1e-5, 'Initial learning rate')

In [2]:
def inference(images_placeholder, keep_prob):
    """ 予測モデルを作成する関数

    引数: 
      images_placeholder: 画像のplaceholder
      keep_prob: dropout率のplace_holder

    返り値:
      y_conv: 各クラスの確率(のようなもの)
    """
    # 重みを標準偏差0.1の正規分布で初期化
    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial)

    # バイアスを標準偏差0.1の正規分布で初期化
    def bias_variable(shape):
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial)

    # 畳み込み層の作成
    def conv2d(x, W):
        return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

    # プーリング層の作成
    def max_pool_2x2(x):
        return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1], padding='SAME')
    
    # 入力を28x28x3に変形
    x_image = tf.reshape(images_placeholder, [-1, 28, 28, 3])

    # 畳み込み層1の作成
    with tf.name_scope('conv1') as scope:
        W_conv1 = weight_variable([5, 5, 3, 32])
        b_conv1 = bias_variable([32])
        h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

    # プーリング層1の作成
    with tf.name_scope('pool1') as scope:
        h_pool1 = max_pool_2x2(h_conv1)
    
    # 畳み込み層2の作成
    with tf.name_scope('conv2') as scope:
        W_conv2 = weight_variable([5, 5, 32, 64])
        b_conv2 = bias_variable([64])
        h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

    # プーリング層2の作成
    with tf.name_scope('pool2') as scope:
        h_pool2 = max_pool_2x2(h_conv2)

    # 全結合層1の作成
    with tf.name_scope('fc1') as scope:
        W_fc1 = weight_variable([7*7*64, 1024])
        b_fc1 = bias_variable([1024])
        h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
        # dropoutの設定
        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    # 全結合層2の作成
    with tf.name_scope('fc2') as scope:
        W_fc2 = weight_variable([1024, NUM_CLASSES])
        b_fc2 = bias_variable([NUM_CLASSES])

    # ソフトマックス関数による正規化
    with tf.name_scope('softmax') as scope:
        y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

    # 各ラベルの確率のようなものを返す
    return y_conv


In [3]:
def loss(logits, labels):
    """ lossを計算する関数

    引数:
      logits: ロジットのtensor, float - [batch_size, NUM_CLASSES]
      labels: ラベルのtensor, int32 - [batch_size, NUM_CLASSES]

    返り値:
      cross_entropy: 交差エントロピーのtensor, float

    """

    # 交差エントロピーの計算
    cross_entropy = -tf.reduce_sum(labels*tf.log(logits))
    # TensorBoardで表示するよう指定
    tf.summary.scalar("cross_entropy", cross_entropy)
    return cross_entropy

In [4]:
def training(loss, learning_rate):
    """ 訓練のOpを定義する関数

    引数:
      loss: 損失のtensor, loss()の結果
      learning_rate: 学習係数

    返り値:
      train_step: 訓練のOp

    """

    train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    return train_step

In [5]:
def accuracy(logits, labels):
    """ 正解率(accuracy)を計算する関数

    引数: 
      logits: inference()の結果
      labels: ラベルのtensor, int32 - [batch_size, NUM_CLASSES]

    返り値:
      accuracy: 正解率(float)

    """
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    tf.summary.scalar("accuracy", accuracy)
    return accuracy

In [6]:
def get_image_and_label(sess, csv_lines):
    images = []
    labels = []
    for train in csv_lines:
        print(train[0])
        print(train[1])
        val = tf.cast(train[0], dtype=tf.string)
        jpeg_r = tf.read_file(val)
        image = tf.image.decode_jpeg(jpeg_r, channels=3)
        image = tf.image.resize_images(image, [28, 28])
        image = tf.reshape(image, [-1])
        # 一列にした後、0-1のfloat値にする
        image_val = sess.run(image).astype(np.float32) / 255.0
        images.append(image_val)
        
        # ラベルを1-of-k方式で用意する
        tmp = np.zeros(NUM_CLASSES)
        tmp[int(train[1])] = 1
        labels.append(tmp)

    return np.asarray(images, dtype=np.float32), np.asarray(labels, dtype=np.float32)

if __name__ == '__main__':
    raw_input = np.loadtxt(open(FLAGS.train), delimiter=",", dtype=str)

#     get path 
#     img_path = Path(FLAGS.train)
#     data = []
#     for path in img_path.iterdir():
#         data.append(str(path))

#     random.shuffle(data)
#     trains = np.array(data[:int(len(data) * .8)])
#     tests = np.array(data[:int(len(data) * .2)])
#     # should change
    [trains, tests] = np.vsplit(raw_input, [TRAIN_SECTION])

    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        train_image, train_label = get_image_and_label(sess, trains)
        test_image, test_label = get_image_and_label(sess, tests)
    
    with tf.Graph().as_default():
        # 画像を入れる仮のTensor
        images_placeholder = tf.placeholder("float", shape=(None, IMAGE_PIXELS))
        # ラベルを入れる仮のTensor
        labels_placeholder = tf.placeholder("float", shape=(None, NUM_CLASSES))
        # dropout率を入れる仮のTensor
        keep_prob = tf.placeholder("float")

        # inference()を呼び出してモデルを作る
        logits = inference(images_placeholder, keep_prob)
        # loss()を呼び出して損失を計算
        loss_value = loss(logits, labels_placeholder)
        # training()を呼び出して訓練
        train_op = training(loss_value, FLAGS.learning_rate)
        # 精度の計算
        acc = accuracy(logits, labels_placeholder)

        # 保存の準備
        saver = tf.train.Saver()
        # Sessionの作成
        sess = tf.Session()
        # 変数の初期化
        sess.run(tf.initialize_all_variables())
        # TensorBoardで表示する値の設定
        summary_op = tf.summary.merge_all()
        summary_writer = tf.summary.FileWriter(FLAGS.train_dir, sess.graph_def)
        
        # 訓練の実行
        for step in range(FLAGS.max_steps):
            for i in range(int(len(train_image)/FLAGS.batch_size)):
                # batch_size分の画像に対して訓練の実行
                batch = FLAGS.batch_size*i
                # feed_dictでplaceholderに入れるデータを指定する
                sess.run(train_op, feed_dict={
                  images_placeholder: train_image[batch:batch+FLAGS.batch_size],
                  labels_placeholder: train_label[batch:batch+FLAGS.batch_size],
                  keep_prob: 0.5})

            # 1 step終わるたびに精度を計算する
            train_accuracy = sess.run(acc, feed_dict={
                images_placeholder: train_image,
                labels_placeholder: train_label,
                keep_prob: 1.0})
            print ("step %d, training accuracy %g"%(step, train_accuracy))

            # 1 step終わるたびにTensorBoardに表示する値を追加する
            summary_str = sess.run(summary_op, feed_dict={
                images_placeholder: train_image,
                labels_placeholder: train_label,
                keep_prob: 1.0}) # Dropout の引数
            summary_writer.add_summary(summary_str, step)

    # 訓練が終了したらテストデータに対する精度を表示
    print ("test accuracy %g"%sess.run(acc, feed_dict={
        images_placeholder: test_image,
        labels_placeholder: test_label,
        keep_prob: 1.0}))

    # 最終的なモデルを保存
    save_path = saver.save(sess, "src/data/preprocess/model.ckpt")

NotADirectoryError: [Errno 20] Not a directory: 'src/data/preprocess/data_aug/train.csv'

In [7]:
# modify ver

def get_image_and_label(sess, csv_lines):
    images = []
    labels = []
    for train in csv_lines:
        print(train[0])
        print(train[1])
        val = tf.cast(train[0], dtype=tf.string)
        jpeg_r = tf.read_file(val)
        image = tf.image.decode_jpeg(jpeg_r, channels=3)
        image = tf.image.resize_images(image, [28, 28])
        image = tf.reshape(image, [-1])
        # 一列にした後、0-1のfloat値にする
        image_val = sess.run(image).astype(np.float32) / 255.0
        images.append(image_val)
        
        # ラベルを1-of-k方式で用意する
        tmp = np.zeros(NUM_CLASSES)
        tmp[int(train[1])] = 1
        labels.append(tmp)

    return np.asarray(images, dtype=np.float32), np.asarray(labels, dtype=np.float32)

if __name__ == '__main__':
#     raw_input = np.loadtxt(open(FLAGS.train), delimiter=",", dtype=str)

    #＃get path 
    img_path = Path(FLAGS.train)
    data = []
    for path in img_path.iterdir():
        data.append(str(path))

    random.shuffle(data)
    trains = np.array(data[:int(len(data) * .8)])
    tests = np.array(data[int(len(data) * .2):])

    # put on label for trains
    trains = []
    tests = []
    for i in trains:
        trains = np.append(i, np.array([1]))
        trains.append(trains)
        tests = np.append(i, np.array([0]))
        tests.append((tests))
#     # should change
#     [trains, tests] = np.vsplit(raw_input, [TRAIN_SECTION])

    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        train_image, train_label = get_image_and_label(sess, trains)
        test_image, test_label = get_image_and_label(sess, tests)
    
    with tf.Graph().as_default():
        # 画像を入れる仮のTensor
        images_placeholder = tf.placeholder("float", shape=(None, IMAGE_PIXELS))
        # ラベルを入れる仮のTensor
        labels_placeholder = tf.placeholder("float", shape=(None, NUM_CLASSES))
        # dropout率を入れる仮のTensor
        keep_prob = tf.placeholder("float")

        # inference()を呼び出してモデルを作る
        logits = inference(images_placeholder, keep_prob)
        # loss()を呼び出して損失を計算
        loss_value = loss(logits, labels_placeholder)
        # training()を呼び出して訓練
        train_op = training(loss_value, FLAGS.learning_rate)
        # 精度の計算
        acc = accuracy(logits, labels_placeholder)

        # 保存の準備
        saver = tf.train.Saver()
        # Sessionの作成
        sess = tf.Session()
        # 変数の初期化
        sess.run(tf.initialize_all_variables())
        # TensorBoardで表示する値の設定
        summary_op = tf.summary.merge_all()
        summary_writer = tf.summary.FileWriter(FLAGS.train_dir, sess.graph_def)
        
        # 訓練の実行
        for step in range(FLAGS.max_steps):
            for i in range(int(len(train_image)/FLAGS.batch_size)):
                # batch_size分の画像に対して訓練の実行
                batch = FLAGS.batch_size*i
                # feed_dictでplaceholderに入れるデータを指定する
                sess.run(train_op, feed_dict={
                  images_placeholder: train_image[batch:batch+FLAGS.batch_size],
                  labels_placeholder: train_label[batch:batch+FLAGS.batch_size],
                  keep_prob: 0.5})

            # 1 step終わるたびに精度を計算する
            train_accuracy = sess.run(acc, feed_dict={
                images_placeholder: train_image,
                labels_placeholder: train_label,
                keep_prob: 1.0})
            print ("step %d, training accuracy %g"%(step, train_accuracy))
            print(type(train_accuracy))

            # 1 step終わるたびにTensorBoardに表示する値を追加する
            summary_str = sess.run(summary_op, feed_dict={
                images_placeholder: train_image,
                labels_placeholder: train_label,
                keep_prob: 1.0}) # Dropout の引数
            summary_writer.add_summary(summary_str, step)

    # 訓練が終了したらテストデータに対する精度を表示
    print ("test accuracy %g"%sess.run(acc, feed_dict={
        images_placeholder: test_image,
        labels_placeholder: test_label,
        keep_prob: 1.0}))

    # 最終的なモデルを保存
    save_path = saver.save(sess, "src/data/preprocess/model.ckpt")

NotADirectoryError: [Errno 20] Not a directory: 'src/data/preprocess/data_aug/train.csv'

In [4]:
import numpy as np
from  pathlib import Path
train = 'src/data/preprocess/data_aug'
img_path = Path(train)

data = []
for path in img_path.iterdir():
    if path.suffix == '.jpg':
        data.append(str(path))
# add label
for i in data:
    trains = np.append(i, np.array([1]))
    tests = np.append(i, np.array([0]))

# # cant random after that fix
# pre_trains = np.array(data[:int(len(data) * .8)])
# pre_tests = np.array(data[int(len(data) * .2):])

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(data, train_size=0.8, random_state=1)

print(X_train)
# put on label for trains

# for i in pre_trains
# trains = np.append(i, np.array([1]))
# tests = np.append(i, np.array([0]))

# print(trains[0])
# print(trains[1])
# print("===")
# print(tests)

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


ValueError: not enough values to unpack (expected 4, got 2)